<a href="https://colab.research.google.com/github/rishubhkhurana/GNN/blob/master/notebooks/DGL101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Downloading Libraries


In [ ]:
!pip install dgl-cu101
!pip install ogb

## Importing Libs

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import dgl
import dgl.nn as gnn
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
dgl.load_backend('pytorch')
from torch.utils.data import DataLoader
import attr

Using backend: pytorch
Using backend: pytorch


## Practice DGL

In [ ]:
_activations = {'elu':nn.ELU(),'relu':nn.ReLU(),'leaky':nn.LeakyReLU()}
class SAGEConvModel(nn.Module):
    def __init__(self,in_channels,gnn_filters, out_channels,aggregator_type='gcn',drop_p=0,activation='relu'):
        super().__init__()
        self.activation = _activations[activation]
        self.gnn_lyrs = nn.ModuleList()
        for i,filters in enumerate(gnn_filters):
            if i==0:
                self.gnn_lyrs.append(gnn.SAGEConv(in_channels,filters,aggregator_type=aggregator_type,feat_drop=drop_p,activation=self.activation))
            else:
                self.gnn_lyrs.append(gnn.SAGEConv(gnn_filters[i-1],filters,aggregator_type=aggregator_type,feat_drop=drop_p,activation=self.activation))
        self.gnn_lyrs.append(gnn.SAGEConv(gnn_filters[-1],out_channels,aggregator_type=aggregator_type,feat_drop=drop_p,activation=None))
    def forward(self,G,features):
        h = features
        for l in self.gnn_lyrs:
            h = l(G,h)
        return h

In [ ]:
_activations = {'elu':nn.ELU(),'relu':nn.ReLU(),'leaky':nn.LeakyReLU()}
class GCNModel(nn.Module):
    def __init__(self,in_channels,gnn_filters, out_channels,aggregator_type='gcn',drop_p=0,activation='relu'):
        super().__init__()
        self.activation = _activations[activation]
        self.gnn_lyrs = nn.ModuleList()
        for i,filters in enumerate(gnn_filters):
            if i==0:
                self.gnn_lyrs.append(gnn.GraphConv(in_channels,filters,activation=self.activation))
            else:
                self.gnn_lyrs.append(gnn.GraphConv(gnn_filters[i-1],filters,activation=self.activation))
        self.gnn_lyrs.append(gnn.GraphConv(gnn_filters[-1],out_channels,activation=None))
    def forward(self,G,features):
        h = features
        for l in self.gnn_lyrs:
            h = l(G,h)
        return h

In [ ]:
model = SAGEConvModel(10,[16,16],4)

In [ ]:
model

SAGEConvModel(
  (activation): ReLU()
  (gnn_lyrs): ModuleList(
    (0): SAGEConv(
      (feat_drop): Dropout(p=0, inplace=False)
      (activation): ReLU()
      (fc_neigh): Linear(in_features=10, out_features=16, bias=True)
    )
    (1): SAGEConv(
      (feat_drop): Dropout(p=0, inplace=False)
      (activation): ReLU()
      (fc_neigh): Linear(in_features=16, out_features=16, bias=True)
    )
    (2): SAGEConv(
      (feat_drop): Dropout(p=0, inplace=False)
      (fc_neigh): Linear(in_features=16, out_features=4, bias=True)
    )
  )
)

## Reading Data

In [ ]:
## reading data
from dgl.data import citegrh

In [ ]:
data = citegrh.load_pubmed()

Finished data loading and preprocessing.
  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000


In [ ]:
len(data)

1

In [ ]:
graph=data[0]

In [ ]:
features = torch.tensor(data.features)
labels = torch.tensor(data.labels)
num_classes = data.num_labels
in_channels = features.shape[1]

In [ ]:
print(f"Number of nodes --> {data[0].number_of_nodes()}")
print(f"Number of edges --> {data[0].number_of_edges()}")

Number of nodes --> 19717
Number of edges --> 88651


In [ ]:
g = data[0]
g.ndata['feat'] = features.to('cuda')
g.ndata['labels'] = labels.to('cuda')

## Defining Model

In [ ]:
## model parameters
model_params = {}
model_params['in_channels'] = in_channels
model_params['gnn_filters'] = [32,32,32]
model_params['out_channels'] = num_classes
model_params['drop_p'] = 0.5

## Trainer

In [ ]:
!rm -rf GNN

In [ ]:
!git clone https://github.com/rishubhkhurana/GNN.git

Cloning into 'GNN'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 59 (delta 20), reused 57 (delta 18), pack-reused 0
Unpacking objects: 100% (59/59), done.


In [ ]:
from GNN.training_loops import *

In [ ]:
node_idxs = g.nodes()
train_idxs = node_idxs[data.train_mask==1.]
valid_idxs = node_idxs[data.val_mask==1.]
test_idxs = node_idxs[data.test_mask==1.]

In [ ]:
dls = dict()
dls['train'] = dataloader.DGLNodeLoader(graph=g,shuffle=True,node_idxs=train_idxs)
dls['test'] = dataloader.DGLNodeLoader(graph=g,shuffle=False,node_idxs=test_idxs)
dls['valid'] = dataloader.DGLNodeLoader(graph=g,shuffle=False,node_idxs=valid_idxs)

In [ ]:
model = SAGEConvModel(**model_params).to('cuda')
opt = torch.optim.Adam(model.parameters(),lr=1e-3)
loss_func = nn.CrossEntropyLoss()

NameError: ignored

In [ ]:
packet = training.TrainPacket(model=model,opt=opt,dls=dls,loss_func=loss_func)

In [ ]:
cbs = [training.PreProcessingCallBack(xnames=['graph','features'],mask_name='node_idxs',ynames=['yb'])]
cbs+=[training.TrainRecorderCallBack(metric_funcs=[(metrics.accuracy(),'summable')]),training.PrintStatsCallBack(freq=50)]

In [ ]:
cbh = training.CallBackHandler(cbs=cbs)

In [ ]:
runner = training.Runner(cbh=cbh,trainpacket=packet)

In [ ]:
runner.fit(epochs=500)

In [ ]:
## GCN Model

In [ ]:
model = GCNModel(**model_params).cuda()
opt = torch.optim.Adam(model.parameters(),lr=1e-3)
loss_func = nn.CrossEntropyLoss()

In [ ]:
packet = training.TrainPacket(model=model,opt=opt,dls=dls,loss_func=loss_func)

In [ ]:
cbs = [training.PreProcessingCallBack(xnames=['graph','features'],mask_name='node_idxs',ynames=['yb'])]
cbs+=[training.TrainRecorderCallBack(metric_funcs=[(metrics.accuracy(),'summable')]),training.PrintStatsCallBack(freq=50)]

In [ ]:
cbh = training.CallBackHandler(cbs=cbs)

In [ ]:
runner = training.Runner(cbh=cbh,trainpacket=packet)

In [ ]:
runner.fit(epochs=500)

wg